In [5]:
import import_ipynb
from CustomDataset import ControlsDataset

import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tensorboardX import SummaryWriter

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [6]:
if __name__ == "__main__":
    dataset = ControlsDataset()
    dataset.labels.num_categories = 14
    dataset.labels.transform.categorize(1.05, -1.05, 21)
    #dataset.images.set_grayscale(True)
    dataset.make_dataloaders()
    #dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)


Total training stacks 7
Total validation stacks 2


In [7]:
class ConvNet(nn.Module):
    def __init__(self,color_channels,outputs,dataset):
        super(ConvNet, self).__init__()
        
        self.channels = color_channels
        self.report_period = 20
        self.writer = SummaryWriter()
        self.start_epoch = 0
        self.infotype = dataset.labels.infotype
        self.dataset = dataset
        self.dataloader = dataset.dataloader
        
        
        
        img_size = torch.Size([1, color_channels, 70, 320]) # [batch_size, channels, height, width]
        empty = torch.zeros(img_size)
        
        
        self.convLayers = nn.Sequential(
            nn.Conv2d(color_channels, 24, 5, stride=2), #in_channels, out_channels, kernel_size, 
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.25)
        )
        
        self.FC = nn.Sequential(
            nn.Linear(in_features=64 * 14 * 73, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=30),
            nn.ELU(),
            nn.Linear(in_features=30, out_features=20), #output features was 10
            nn.Linear(in_features=20, out_features=outputs)
        )
        
    
    def forward(self, input):
        
#         '[128, 3, 70, 320]'
#         input = input.view(input.size(0), 3, 70, 320)
#         print("input Shape: ", input.shape)
        output = self.convLayers(input)
        
#         print("ConvNet Shape", output.shape)  # torch.Size([128, 64, 14, 73])
#         output = output.view(output.size(0), -1) # Image flatting
        output = output.view(output.size(0), 64 * 14 * 73)
#         print("Output Shape", output.shape)
        output = self.FC(output)
        
        return output
    
    
    def load(self,path):
        checkpoint = torch.load(path)
        self.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.start_epoch = checkpoint['epoch']
        
    def save(self,epoch,path):
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()}, 
            path)
    
    
    def score(self,device,dataset, single_batch = False):
        self.eval()
        total = 0
        correct = 0
        for i_batch, sampled_batch in enumerate(dataset.dataloader):
            images = sampled_batch['image'].to(device).float()
            controls = sampled_batch['control'].to(device).long()
            controls = torch.flatten(controls)
            prediction = self(images)

            maximum = torch.argmax(prediction,dim = 1)
            shared = maximum == controls
            shared = 1 * shared
            correct += int(torch.sum(shared))
            total += len(controls)
            if single_batch:
                return (total,correct)
        return (total,correct)
    
    
    def fit(self, device, epochs, optimizer, criterion):

        self.train()
        self.optimizer = optimizer
        iter_no = 0

        for epoch in range(self.start_epoch, epochs):
            for iBatch, sampledBatch in enumerate(self.dataloader):

                images = sampledBatch['image'].to(device).float()
                controls = sampledBatch['control'].to(device).long()
                controls = torch.flatten(controls)

                optimizer.zero_grad()

                outputs = self(images)
                
                loss = criterion(outputs, controls)
                print("Loss:", loss)
#                 print(outputs)
#                 print(controls)
            
                loss.backward()
                optimizer.step()

#                 out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"

                score = self.score(device,self.dataset)
                percent = round(score[1]/score[0],3)
                total,correct = score
                
                print("Score: ", score, "Percentage: ", percent, "Total:", total, "Correct:", correct )
                

#                 iter_no += 1
#                 if iter_no % self.report_period == 0:
#                     print("Score: ", score)
#                     print("Percentage: ", percent)
#                     print("Total:", total, "Correct:", correct)
#                     self.writer.add_scalar("Loss", loss.item(), iter_no)
#                     print("saved to tensorboard")
#                     self.save(epoch,"snapshots/{:.3f}_model.pt".format(loss.item()))


                print("IN THIS BATCH")
            print("Completed Epoch", epoch)

                
        
        
        
    

In [4]:
if __name__ == "__main__":
    if dataset.images.grayscale:
        channels = 1
    else:
        channels = 3

    net = ConvNet(channels, 21, dataset)